# Repairs of Python.NET 3 breaking changes to internal tests

## Load a runtime before calling `import clr`


In order to access .NET assemblies (`.dll` files), one must load an available runtime before executing the
`import clr` statement. (If one calls `import clr` before specifying a runtime, Python.NET will load a default
runtime which may **not** be compatible with the installed Orchid assemblies.

To make this easier, when we `import` the `orchid` package, the Orchid Python API will load the runtime
corresponding to the configured Orchid installation.

In [ ]:
import orchid

In [ ]:
#noinspection PyPackageRequirements
import clr

In [ ]:
# noinspection PyUnresolvedReferences,PyPackageRequirements
from System import (
    ArgumentException,
    Convert,
    DateTime,
    DateTimeKind,
    DateTimeOffset,
    Int32,
    InvalidCastException,
    TimeSpan,
)

## Reduce the implicit conversions between Python types and .NET Types

### Equality between Python `int` values and `DateTimeOffset.MaxValue` no longer supported

Python.NET 2.5.2 allowed a developer to test for equality between Python `int` values and values of type `DateTimeOffset`
by performing an implicit conversion of the `DateTimeOffset` type.

In [ ]:
try:
    print('Trying `108 == DateTimeOffset.Value`')
    108 == DateTimeOffset.MaxValue
except TypeError as nie:
    print(f'TypeError: {nie}')

In [ ]:
try:
    print('Trying `108 == TimeSpan.MaxValue`')
    108 == TimeSpan.MaxValue
except TypeError as nie:
    print(f'TypeError: {nie}')

We filed an issue with the Python.NET team. They responded with the following:

> Yes, we tried to limit the "implicit" conversions to a minimum. I don't even know, which change in particular is responsible for the behavioural ~change~ fix that you are observing here, but you are only able to compare things to a .NET object that are directly convertible to it. If you'd really require this for `DateTimeOffset` and `TimeSpan`, you could make them convertible via a [Codec](https://pythonnet.github.io/pythonnet/codecs.html). Otherwise, I'd suggest you just generate the respective comparison values using `.FromTicks`.

Python.NET 3 requires a developer to explicitly use the `DateTimeOffset.Ticks` for in the equality test.

In [ ]:
108 == DateTimeOffset.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MinValue.Ticks

### Less effort to make Python constructors "just work"

Python.NET 2.5.2 allowed expressions like `TimeSpan()`. (Note that the .NET `TimeSpan` class **does not** have a 
default constructor.) This expression is no longer supported. Instead, one must supply an argument 
(typically zero (0)) to the constructor or to methods like `TimeSpan.FromTicks()`.

In [ ]:
try:
    print('Trying expression, `TimeSpan()`')
    TimeSpan()
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
TimeSpan(8801).ToString()

In [ ]:
TimeSpan(0).ToString()

In [ ]:
TimeSpan.FromTicks(0).ToString()

## Adding attributes with integer values requires explicit conversion

(This issue occurred in **both** internal testing and low-level script testing and so is duplicated.)

During integration testing, we discovered an issue setting an attribute with type, `Int32`, using a Python `int` 
value of 7. The run-time reported that the types, `Int32` and `PyInt` were incompatible. 

This scenario requires significant set up.

In [13]:
# Find the well named 'Demo_1H'
bakken = orchid.load_project('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac')
candidate_wells = list(bakken.wells().find_by_name('Demo_1H'))
assert len(candidate_wells) == 1
demo_1h = candidate_wells[0]

In [14]:
# Create an attribute with name, 'My New Attribute', and type, `System.Int32`
# noinspection PyUnresolvedReferences,PyPackageRequirements
from Orchid.FractureDiagnostics.Factories.Implementations import Attribute

attribute_to_add_type = Int32
attribute_to_add = Attribute[attribute_to_add_type].Create('My New Attribute')

In [15]:
# Add newly created attribute to well, 'Demo_1H'
with orchid.dot_net_disposable.disposable(demo_1h.dom_object.ToMutable()) as mutable_well:
    mutable_well.AddStageAttribute(attribute_to_add)

In [16]:
# Find stage number 7 in well, 'Demo_1H'
maybe_stage = demo_1h.stages().find_by_display_stage_number(7)
assert maybe_stage is not None
stage_7 = maybe_stage

In [17]:
# Add attribute with value, 17, to stage 7, with Python `int` type.
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    # This action will fail because the attribute type is `System.Int32`
    # and `pythonnet-3.0.0.post1` **does not** implicitly equate these two types.
    try:
        mutable_stage.SetAttribute(attribute_to_add, int)
    except ArgumentException as ae:
        print(f'ArgumentException: {ae}')


ArgumentException: Attribute type System.Int32 does not match value type Python.Runtime.PyObject
   at Orchid.FractureDiagnostics.Factories.Implementations.Stage.SetAttributeValue(IAttribute attribute, Object value) in C:\src\OrchidApp\Orchid\Orchid.FractureDiagnostics.Factories\Implementations\Stage.cs:line 690
   at Orchid.FractureDiagnostics.Factories.Implementations.MutableStage.SetAttribute(IAttribute attribute, Object value) in C:\src\OrchidApp\Orchid\Orchid.FractureDiagnostics.Factories\Implementations\Stage.cs:line 805


In [18]:
# Add attribute to stage 7 with a value of 17 **explicitly** converted to an `Int32`
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    mutable_stage.SetAttribute(attribute_to_add, attribute_to_add_type(7))

In [19]:
# Verify added attribute value
ignored_object = object()
is_attribute_present, actual_attribute_value = stage_7.dom_object.TryGetAttributeValue(attribute_to_add,
                                                                                       ignored_object)
assert is_attribute_present
assert type(actual_attribute_value) == int
assert actual_attribute_value == 7

## Disabled implicit conversion from C# Enums to Python `int` and back

### Reduced need for use of `Overloads` (`__overloads__` in Python.NET 3)

The .NET `DateTime` class has many overloaded constructors. Because version Python.NET 2.5.2 converted members 
of .NET Enum types into Python `int` values, the method resolution process could not distinguish between the
`DateTime` constructor taking 7 `System.Int32` arguments (the last specifying milliseconds) and the constructor
accepting 6 `System.Int32` values and a `DateTimeKind` member. Consequently, a developer of the Orchid Python API
had to specify an overload in order to invoke the appropriate constructor.

In [20]:
DateTime

System.DateTime

In [21]:
# Querying the `__overloads__` (preferred but `Overloads` is also available) produces a very unexpected result.
#
# Our working hypothesis is that the Python.NET method resolution algorithm could find any overloads for the 
# constructor resulting in 
DateTime.Overloads, DateTime.__overloads__

(<unbound method '__init__'>, <unbound method '__init__'>)

In [22]:
type(DateTime.__overloads__)

CLR.MethodBinding

In [23]:
dir(DateTime)

['Add',
 'AddDays',
 'AddHours',
 'AddMilliseconds',
 'AddMinutes',
 'AddMonths',
 'AddSeconds',
 'AddTicks',
 'AddYears',
 'Compare',
 'CompareTo',
 'Date',
 'Day',
 'DayOfWeek',
 'DayOfYear',
 'DaysInMonth',
 'Equals',
 'Finalize',
 'FromBinary',
 'FromFileTime',
 'FromFileTimeUtc',
 'FromOADate',
 'GetDateTimeFormats',
 'GetHashCode',
 'GetType',
 'GetTypeCode',
 'Hour',
 'IsDaylightSavingTime',
 'IsLeapYear',
 'Kind',
 'MaxValue',
 'MemberwiseClone',
 'Millisecond',
 'MinValue',
 'Minute',
 'Month',
 'Now',
 'Overloads',
 'Parse',
 'ParseExact',
 'ReferenceEquals',
 'Second',
 'SpecifyKind',
 'Subtract',
 'Ticks',
 'TimeOfDay',
 'ToBinary',
 'ToFileTime',
 'ToFileTimeUtc',
 'ToLocalTime',
 'ToLongDateString',
 'ToLongTimeString',
 'ToOADate',
 'ToShortDateString',
 'ToShortTimeString',
 'ToString',
 'ToUniversalTime',
 'Today',
 'TryParse',
 'TryParseExact',
 'UtcNow',
 'Year',
 '__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '

In [24]:
type(DateTimeKind.Utc)

System.DateTimeKind

In [25]:
dir(DateTimeKind.Utc)

['CompareTo',
 'Equals',
 'Finalize',
 'Format',
 'GetHashCode',
 'GetName',
 'GetNames',
 'GetType',
 'GetTypeCode',
 'GetUnderlyingType',
 'GetValues',
 'HasFlag',
 'IsDefined',
 'Local',
 'MemberwiseClone',
 'Overloads',
 'Parse',
 'ReferenceEquals',
 'ToObject',
 'ToString',
 'TryParse',
 'Unspecified',
 'Utc',
 '__and__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__float__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__overloads__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__xor__',
 'value__']

In [26]:
DateTimeKind.Utc.GetType()

In [27]:
dir(DateTimeKind.Utc.GetType())

['AsType',
 'Assembly',
 'AssemblyQualifiedName',
 'Attributes',
 'BaseType',
 'Clone',
 'ContainsGenericParameters',
 'CustomAttributes',
 'DeclaredConstructors',
 'DeclaredEvents',
 'DeclaredFields',
 'DeclaredMembers',
 'DeclaredMethods',
 'DeclaredNestedTypes',
 'DeclaredProperties',
 'DeclaringMethod',
 'DeclaringType',
 'DefaultBinder',
 'Delimiter',
 'EmptyTypes',
 'Equals',
 'FilterAttribute',
 'FilterName',
 'FilterNameIgnoreCase',
 'Finalize',
 'FindInterfaces',
 'FindMembers',
 'FullName',
 'GUID',
 'GenericParameterAttributes',
 'GenericParameterPosition',
 'GenericTypeArguments',
 'GenericTypeParameters',
 'GetArrayRank',
 'GetAttributeFlagsImpl',
 'GetConstructor',
 'GetConstructorImpl',
 'GetConstructors',
 'GetCustomAttributes',
 'GetCustomAttributesData',
 'GetDeclaredEvent',
 'GetDeclaredField',
 'GetDeclaredMethod',
 'GetDeclaredMethods',
 'GetDeclaredNestedType',
 'GetDeclaredProperty',
 'GetDefaultMembers',
 'GetElementType',
 'GetEnumName',
 'GetEnumNames',
 'GetE

In [28]:
DateTimeKind.Utc.GetType().BaseType

In [29]:
DateTimeKind.Utc.GetType().BaseType.FullName

'System.Enum'

In [30]:
DateTime(2021, 12, 1, 12, 15, 37, DateTimeKind.Utc).ToString('o')

'2021-12-01T12:15:37.0000000Z'

### Eliminated need to inherit from Python `enum.IntEnum` for compatibility with .NET Enum types

Version 2.5.2 of `pythonnet` converted values of type .NET Enum to Python `int` values. Consequently, to support 
easy comparisons between the .NET type, `Orchid.FractureDiagnostics.FormationConnectionType` and the Python 
enumeration, `native_stage_adapter.ConnectionType`, we defined `native_stage_adapter.ConnectionType` to inherit 
from `enum.IntEnum`.  This base class is not needed in `pythonnet-3.0.0.post1` because the enumeration member 
`native_stage_adatper.ConnectionType.PLUG_AND_PERF`, defined to have a value of 
`Orchid.FractureDiagnostics.FormationConnectionType` is no longer of type `int` but is actually of type, 
`Orchid.FractureDiagnostics.FormationConnectionType`.

In [31]:
# Returned `True` in `pythonnet-2.5.2`
orchid.net_date_time.TimePointTimeZoneKind.UTC == 0

False

In [32]:
orchid.net_date_time.DateTimeKind

System.DateTimeKind

In [33]:
orchid.net_date_time.TimePointTimeZoneKind

<enum 'TimePointTimeZoneKind'>

In [34]:
orchid.net_date_time.TimePointTimeZoneKind.UTC

<TimePointTimeZoneKind.UTC: <System.DateTimeKind object at 0x0000018BF22B8BC0>>

In [35]:
orchid.net_date_time.TimePointTimeZoneKind.UTC.value

In [36]:
# Similarly, this expression returned `True` in `pythonnet-2.5.2`
orchid.native_stage_adapter.ConnectionType == 0

False

In [37]:
orchid.native_stage_adapter.FormationConnectionType

Orchid.FractureDiagnostics.FormationConnectionType

In [38]:
orchid.native_stage_adapter.ConnectionType

<enum 'ConnectionType'>

In [39]:
orchid.native_stage_adapter.ConnectionType.PLUG_AND_PERF

<ConnectionType.PLUG_AND_PERF: <Orchid.FractureDiagnostics.FormationConnectionType object at 0x0000018BF357F680>>

In [40]:
orchid.native_stage_adapter.ConnectionType.OPEN_HOLE

<ConnectionType.OPEN_HOLE: <Orchid.FractureDiagnostics.FormationConnectionType object at 0x0000018BF357F780>>

In [41]:
orchid.native_stage_adapter.ConnectionType.PLUG_AND_PERF.value

## Return values from .NET methods that return an interface are now automatically wrapped in that interface

Under `pythonnet-2.5.2`, running the following `doctest` passes:

```
    >>> start = pendulum.parse('2022-02-23T15:53:23Z')
    >>> stop = pendulum.parse('2022-02-24T05:54:11Z')
    >>> net_start = ndt.as_net_date_time(start)
    >>> net_stop = ndt.as_net_date_time(stop)
    >>> factory = create()
    >>> date_time_offset_range = factory.CreateDateTimeOffsetRange(net_start, net_stop)
    >>> (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
    ('2022-02-23T15:53:23.0000000+00:00', '2022-02-24T05:54:11.0000000+00:00')
```

When running the same `doctest` using `pythonnet-3.0.0.post1`, this code
encounters an unhandled exception:

```
Error
**********************************************************************
File "C:\src\orchid-python-api\orchid\net_fracture_diagnostics_factory.py", line ?, in net_fracture_diagnostics_factory.create
Failed example:
    (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
Exception raised:
    Traceback (most recent call last):
      File "C:/Users/larry.jones/AppData/Local/JetBrains/Toolbox/apps/PyCharm-P/ch-0/222.4459.20/plugins/python/helpers/pycharm/docrunner.py", line 138, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest net_fracture_diagnostics_factory.create[6]>", line 1, in <module>
        (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
    TypeError: No method matches given arguments for Object.ToString: (<class 'str'>)
```

In [42]:
import pendulum

In [43]:
start = pendulum.parse('2022-02-23T15:53:23Z')
stop = pendulum.parse('2022-02-24T05:54:11Z')
net_start = orchid.net_date_time.as_net_date_time(start)
net_stop = orchid.net_date_time.as_net_date_time(stop)
factory = orchid.net_fracture_diagnostics_factory.create()

In [44]:
type(start), type(stop), type(net_start), type(net_stop), type(factory)

(pendulum.datetime.DateTime,
 pendulum.datetime.DateTime,
 System.DateTime,
 System.DateTime,
 Orchid.FractureDiagnostics.IFractureDiagnosticsFactory)

In [45]:
date_time_offset_range = factory.CreateDateTimeOffsetRange(net_start, net_stop)
type(date_time_offset_range)

Orchid.FractureDiagnostics.IDateTimeOffsetRange

In [46]:
dir(date_time_offset_range)

['Equals',
 'GetHashCode',
 'GetType',
 'IsInRange',
 'Start',
 'Stop',
 'ToString',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'get_Start',
 'get_Stop']

In [47]:
str(date_time_offset_range)

'Orchid.FractureDiagnostics.Factories.Implementations.DateTimeOffsetRange'

In [48]:
date_time_offset_range.Start.GetType().FullName

'System.DateTimeOffset'

In [49]:
net_range_start = date_time_offset_range.Start
type(net_range_start)

System.IComparable

In [50]:
try:
    net_range_start.ToString('o')
except TypeError as te:
    print(f'TypeError: {te}')

TypeError: No method matches given arguments for Object.ToString: (<class 'str'>)


In [51]:
net_range_start.ToString()

'2/23/2022 3:53:23 PM +00:00'

In [52]:
net_range_start.ToString.__doc__

'System.String ToString()'

In [53]:
dir(DateTimeOffset)

['Add',
 'AddDays',
 'AddHours',
 'AddMilliseconds',
 'AddMinutes',
 'AddMonths',
 'AddSeconds',
 'AddTicks',
 'AddYears',
 'Compare',
 'CompareTo',
 'Date',
 'DateTime',
 'Day',
 'DayOfWeek',
 'DayOfYear',
 'Equals',
 'EqualsExact',
 'Finalize',
 'FromFileTime',
 'FromUnixTimeMilliseconds',
 'FromUnixTimeSeconds',
 'GetHashCode',
 'GetType',
 'Hour',
 'LocalDateTime',
 'MaxValue',
 'MemberwiseClone',
 'Millisecond',
 'MinValue',
 'Minute',
 'Month',
 'Now',
 'Offset',
 'Overloads',
 'Parse',
 'ParseExact',
 'ReferenceEquals',
 'Second',
 'Subtract',
 'Ticks',
 'TimeOfDay',
 'ToFileTime',
 'ToLocalTime',
 'ToOffset',
 'ToString',
 'ToUniversalTime',
 'ToUnixTimeMilliseconds',
 'ToUnixTimeSeconds',
 'TryParse',
 'TryParseExact',
 'UtcDateTime',
 'UtcNow',
 'UtcTicks',
 'Year',
 '__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__modu

In [54]:
net_date_time_offset = DateTimeOffset.UtcNow
net_date_time_offset

In [55]:
str(net_date_time_offset)

'12/3/2022 11:43:05 PM +00:00'

In [56]:
repr(net_date_time_offset)

'<System.DateTimeOffset object at 0x0000018B90170880>'

In [57]:
net_date_time_offset.ToString()

'12/3/2022 11:43:05 PM +00:00'

In [58]:
net_date_time_offset.ToString('o')

'2022-12-03T23:43:05.9968580+00:00'

In [59]:
type(net_date_time_offset)

System.DateTimeOffset

In [60]:
type(net_range_start)

System.IComparable

In [61]:
net_range_start.GetType().FullName

'System.DateTimeOffset'

In [62]:
dir(net_range_start.GetType())

['AsType',
 'Assembly',
 'AssemblyQualifiedName',
 'Attributes',
 'BaseType',
 'Clone',
 'ContainsGenericParameters',
 'CustomAttributes',
 'DeclaredConstructors',
 'DeclaredEvents',
 'DeclaredFields',
 'DeclaredMembers',
 'DeclaredMethods',
 'DeclaredNestedTypes',
 'DeclaredProperties',
 'DeclaringMethod',
 'DeclaringType',
 'DefaultBinder',
 'Delimiter',
 'EmptyTypes',
 'Equals',
 'FilterAttribute',
 'FilterName',
 'FilterNameIgnoreCase',
 'Finalize',
 'FindInterfaces',
 'FindMembers',
 'FullName',
 'GUID',
 'GenericParameterAttributes',
 'GenericParameterPosition',
 'GenericTypeArguments',
 'GenericTypeParameters',
 'GetArrayRank',
 'GetAttributeFlagsImpl',
 'GetConstructor',
 'GetConstructorImpl',
 'GetConstructors',
 'GetCustomAttributes',
 'GetCustomAttributesData',
 'GetDeclaredEvent',
 'GetDeclaredField',
 'GetDeclaredMethod',
 'GetDeclaredMethods',
 'GetDeclaredNestedType',
 'GetDeclaredProperty',
 'GetDefaultMembers',
 'GetElementType',
 'GetEnumName',
 'GetEnumNames',
 'GetE

In [63]:
[iface.FullName for iface in net_range_start.GetType().GetInterfaces()]

['System.IComparable',
 'System.IFormattable',
 'System.Runtime.Serialization.ISerializable',
 'System.Runtime.Serialization.IDeserializationCallback',
 'System.IComparable`1[[System.DateTimeOffset, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]]',
 'System.IEquatable`1[[System.DateTimeOffset, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]]']

In [64]:
try:
    error_net_range_start = Convert.ChangeType(date_time_offset_range, DateTimeOffset)
    type(error_net_range_start)
except InvalidCastException as ice:
    print(f'InvalidCastException {ice}')

InvalidCastException Object must implement IConvertible.
   at System.Convert.ChangeType(Object value, Type conversionType, IFormatProvider provider)


In [65]:
clr.AddReference('System.Globalization')
# noinspection PyUnresolvedReferences,PyPackageRequirements
from System.Globalization import DateTimeFormatInfo

In [66]:
dir(DateTimeFormatInfo)

['AMDesignator',
 'AbbreviatedDayNames',
 'AbbreviatedMonthGenitiveNames',
 'AbbreviatedMonthNames',
 'Calendar',
 'CalendarWeekRule',
 'Clone',
 'CurrentInfo',
 'DateSeparator',
 'DayNames',
 'Equals',
 'Finalize',
 'FirstDayOfWeek',
 'FullDateTimePattern',
 'GetAbbreviatedDayName',
 'GetAbbreviatedEraName',
 'GetAbbreviatedMonthName',
 'GetAllDateTimePatterns',
 'GetDayName',
 'GetEra',
 'GetEraName',
 'GetFormat',
 'GetHashCode',
 'GetInstance',
 'GetMonthName',
 'GetShortestDayName',
 'GetType',
 'InvariantInfo',
 'IsReadOnly',
 'LongDatePattern',
 'LongTimePattern',
 'MemberwiseClone',
 'MonthDayPattern',
 'MonthGenitiveNames',
 'MonthNames',
 'NativeCalendarName',
 'Overloads',
 'PMDesignator',
 'RFC1123Pattern',
 'ReadOnly',
 'ReferenceEquals',
 'SetAllDateTimePatterns',
 'ShortDatePattern',
 'ShortTimePattern',
 'ShortestDayNames',
 'SortableDateTimePattern',
 'TimeSeparator',
 'ToString',
 'UniversalSortableDateTimePattern',
 'YearMonthPattern',
 '__class__',
 '__delattr__',
 

In [67]:
try:
    net_range_start.ToString(DateTimeFormatInfo.SortableDateTimePattern)
except TypeError as nie:
    print(f'TypeError: {nie}')

TypeError: No method matches given arguments for Object.ToString: (<class 'CLR.PropertyObject'>)


In [68]:
try:
    orchid.net_date_time.as_date_time(net_range_start)
except NotImplementedError:
    print(f'NotImplementedError')

NotImplementedError


In [69]:
try:
    (DateTimeOffset)(net_range_start)
except TypeError as te:
    print(f'TypeError: {te}')

TypeError: No method matches given arguments for DateTimeOffset..ctor: (<class 'System.IComparable'>)


In [70]:
a_date_time_offset = DateTimeOffset.UtcNow
a_date_time_offset = net_range_start
a_date_time_offset

In [71]:
dir(net_range_start)

['CompareTo',
 'Equals',
 'GetHashCode',
 'GetType',
 'ToString',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [72]:
type(date_time_offset_range)

Orchid.FractureDiagnostics.IDateTimeOffsetRange

In [73]:
dir(date_time_offset_range)

['Equals',
 'GetHashCode',
 'GetType',
 'IsInRange',
 'Start',
 'Stop',
 'ToString',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'get_Start',
 'get_Stop']

In [74]:
net_range_start.GetType().GetElementType()

In [75]:
net_range_start.GetType().GetInterface('DateTimeOffset')

In [76]:
try:
    DateTimeOffset.GetType().IsInstanceOfType(net_range_start)
except TypeError as te:
    print(f'TypeError: {te}')

TypeError: not enough arguments


In [77]:
a_date_time_offset.GetType().IsInstanceOfType(net_range_start)

True

In [78]:
net_range_start.GetType().BaseType.FullName

'System.ValueType'

I have not been able to determine a mechanism to convert or cast the type of `IDateTimeOffsetRange.Start` 
(and `IDateTimeOffsetRange.Stop`) from `IComparable` "down" to the concrete type. Consequently, I will use the 
work around of simply invoking `Object.ToString()` and comparing.

In [79]:
date_time_offset_range.Start.ToString(), date_time_offset_range.Stop.ToString()

('2/23/2022 3:53:23 PM +00:00', '2/24/2022 5:54:11 AM +00:00')

After implementing the `ToString()` (no arguments) work around, I received a [response](https://github.com/pythonnet/pythonnet/issues/2034#issuecomment-1332728831) to the issue that I [posted](https://github.com/pythonnet/pythonnet/issues/2034):

> You can access `__implementation__` (codecs applied) or `__raw_implementaion__` (codecs not applied).

This solution is a much better solution than my work around.

In [80]:
net_range_start.__implementation__

In [81]:
net_range_start.__raw_implementation__

In [82]:
[o.__implementation__.ToString('o') for o in (date_time_offset_range.Start, date_time_offset_range.Stop)]

['2022-02-23T15:53:23.0000000+00:00', '2022-02-24T05:54:11.0000000+00:00']